## Группа DS03-onl

Студент Парфимович Алексей

## Домашнее задание №12

Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)
В частности использовать:

GaussianNB
LinearRegression + PolynomialFeatures.
При обучении использовать make_pipeline
GridSearchCV для поиска параметров


In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt

from math import nan
from datetime import datetime 

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score


DATA_CSV_PATH = 'MFOcredit.csv'

### Загрузка, анализ и преобразование данных

Предварительная загрузка и отображение части данных

In [2]:
df_sample = pd.read_csv(DATA_CSV_PATH, sep=';', nrows=3)
#df_sample.head(3)

Загрузка требуемых данных: для признаков явно указываем требуемые типы, признак id исключаем из набора

In [3]:
dtypes={
    'date_start': object,
    'date_end': object,
    'gender': object,
    'age': int,
    'auto': object,
    'housing': object,
    'marstatus': object,
    'regclient': object,
    'jobtype': object,
    'region': object,
    'credits': object,
    'children': object,
    'delinq60plus': object
    }

# Получить список признаков набора данных
col_list = df_sample.columns.tolist()

# Исключить признак 'id' из выборки
cols = col_list[1:]

df_raw = pd.read_csv(DATA_CSV_PATH, usecols = cols, dtype = dtypes, sep=';')

# Заменить значения "пропуск поля" на NULL/NAN
#df_raw.replace('Пропуск поля', nan, inplace=True)

df_raw.head()

,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,03-Jan-2013,12-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,03-Jan-2013,17-Jan-2013,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,03-Jan-2013,17-Jan-2013,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
3,03-Jan-2013,17-Jan-2013,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
4,03-Jan-2013,17-Jan-2013,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет


Заменить признаки дат начала (date_start) и окончания (date_end) периода займа на интегральный признак срока займа (loan_term)

In [4]:
date_start = pd.to_datetime(df_raw['date_start'])
date_end = pd.to_datetime(df_raw['date_end'])

df_raw.insert(0, 'start_year', date_start.dt.year) #.astype("category")) 
df_raw.insert(1, 'start_quart', date_start.dt.quarter) #.astype("category"))
df_raw.insert(2, 'start_month', date_start.dt.month) #.astype("category"))
df_raw.insert(3, 'start_day', date_start.dt.day) #.astype("category"))

df_raw.insert(4, 'end_year', date_end.dt.year) #.astype("category")) 
df_raw.insert(5, 'end_quart', date_end.dt.quarter) #.astype("category"))
df_raw.insert(6, 'end_month', date_end.dt.month) #.astype("category"))
df_raw.insert(7, 'end_day', date_end.dt.day) #.astype("category"))

df_raw.insert(8,'loan_term', (date_end - date_start).dt.days)

df_raw.drop(['date_start','date_end'], inplace=True, axis=1)
#df_raw.info()

Выделить и удалить строки-дубликаты

In [5]:
#df_duplicateRows = df_raw[df_raw.duplicated()]
#df_duplicateRows.shape

df_raw = df_raw.drop_duplicates(keep='first')
df_raw.shape

(35105, 20)

Выполнить анализ пропущенных значений

In [ ]:
col_list = [i for i in df_raw.columns if df_raw[i].dtype.name=='object']
print('Процент значений "Пропуск поля" в предикторах:')
for col in col_list:
    if df_raw[col].str.contains(r"Пропуск поля").any():
        print(col)
        print(round((df_raw[col].value_counts()['Пропуск поля'])/
                    df_raw.shape[0]*100),'%','\n')
    else:
        continue

Перевести кетегориальные признаки с бинарными значениями в числовую форму (Да=1 , Нет=0)

In [6]:
df_raw['gender'] = df_raw['gender'].apply(lambda x: 1 if x == 'Мужской' else 0 if x == 'Женский' else nan)

#df_raw['auto'] = df_raw['auto'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)

df_raw['regclient'] = df_raw['regclient'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)

#df_raw['jobtype'] = df_raw['jobtype'].apply(lambda x: 1 if x == 'Официальное' else 0 if x == 'Неофициальное' else nan)

df_raw['credits'] = df_raw['credits'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)

df_raw['children'] = df_raw['children'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)

df_raw['delinq60plus'] = df_raw['delinq60plus'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)

df_raw['delinq60plus'].value_counts()

0    21614
1    13491
Name: delinq60plus, dtype: int64

### Замена отсутствующих значений категориальных признаков на наиболее часто встречающееся значение из общей группы

In [7]:
# Для признака "children" - значение из группы по признакам 'gender','age','region'
df_raw['children'] = df_raw.groupby(['gender','age','region'])['children'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
df_raw['children'].value_counts()

0.0    21670
1.0    13435
Name: children, dtype: int64

In [8]:
df_raw['credits'] = df_raw.groupby(['gender','age','region'])['credits'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
df_raw['credits'].value_counts()

0.0    18659
1.0    16445
Name: credits, dtype: int64

In [9]:
df_raw['marstatus'] = df_raw.groupby(['gender','age','region','children'])['marstatus'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
df_raw['marstatus'].value_counts()

Гражданский брак/женат/замужем    14077
Холост                             8819
Пропуск поля                       7497
Разведен                           2870
Вдова/вдовец                       1842
Name: marstatus, dtype: int64

In [10]:
df_raw['housing'] = df_raw.groupby(['gender','age','region'])['housing'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
df_raw['housing'].value_counts()

Собственное              18067
Пропуск поля              7536
Живут с родителями        3172
Долевая собственность     2954
Арендуемое                2018
Муниципальное             1358
Name: housing, dtype: int64

Удалить оставшиеся "пропущенные" (NAN) значения

In [11]:
df_raw.dropna(inplace=True)
df_raw.shape

(35104, 20)

Разбиение признаков на группы числовых и категориальных признаков

In [12]:
num_columns = ['start_year', 'start_quart', 'start_month', 'start_day', 'end_year', 'end_quart', 'end_month', 'end_day', 'loan_term', 'age']
cat_columns = ['gender', 'housing', 'marstatus', 'regclient', 'region', 'credits', 'children', 'delinq60plus']
cat_unknowns = ['auto', 'jobtype']

In [13]:
df_num = df_raw[num_columns]
df_dummed = pd.get_dummies(df_raw[cat_columns])
df_unknowns = pd.get_dummies(df_raw[cat_unknowns])

data = pd.concat((df_num, df_dummed, df_unknowns), axis=1)
data.head()

,start_year,start_quart,start_month,start_day,end_year,end_quart,end_month,end_day,loan_term,age,...,region_Кемеровская область север,region_Кемеровская область юг,region_Красноярский край,region_Новосибирская область,auto_Да,auto_Нет,auto_Пропуск поля,jobtype_Неофициальное,jobtype_Официальное,jobtype_Пропуск поля
0,2013,1,1,3,2013,1,1,12,9,44,...,0,0,0,1,0,1,0,0,1,0
1,2013,1,1,3,2013,1,1,17,14,21,...,0,1,0,0,0,0,1,0,1,0
2,2013,1,1,3,2013,1,1,17,14,25,...,1,0,0,0,0,0,1,0,1,0
3,2013,1,1,3,2013,1,1,17,14,47,...,1,0,0,0,0,0,1,0,1,0
4,2013,1,1,3,2013,1,1,17,14,22,...,1,0,0,0,0,1,0,0,1,0


Выполнить разбиение исходной выборки данных на 2 подмножества (обучение и тестирование)

In [14]:
y = data['delinq60plus']
X = data.drop(['delinq60plus'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

### Выполнить обучение и проверку модели RandomForest

In [17]:
forest = RandomForestClassifier()
                                
forest.fit(X_train, y_train);

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, forest.predict_proba(X_train)[:, 1])))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, forest.predict_proba(X_test)[:, 1])))

AUC на обучающей выборке: 1.000
AUC на контрольной выборке: 0.843


Выполнить подбор следующих гиперпараметров модели RandomForest:

n_estimators — число «деревьев» в «случайном лесу».  
max_features — число признаков для выбора расщепления.  
max_depth — максимальная глубина деревьев.  
min_samples_split — минимальное число объектов, необходимое для того, чтобы узел дерева мог бы расщепиться.  
min_samples_leaf — минимальное число объектов в листьях.  
bootstrap — использование для построения деревьев подвыборки с возвращением.  

In [16]:
params = { 'n_estimators': range (100, 800, 100),
            'max_features': ['log2', 'sqrt'],
            'max_depth': range (10, 18, 2),
            'min_samples_leaf': range (2,9,2),
            'min_samples_split': range (2,9,2),
            'bootstrap': [True, False]}

forest_grid = GridSearchCV(forest, param_grid=params, n_jobs=-1, cv=5, verbose=5)
forest_grid.fit(X_train,y_train)

print('Best Parameters : {}'.format(forest_grid.best_params_))
print('Best Accuracy Through Grid Search : {:.3f}\n'.format(forest_grid.best_score_))

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, forest_grid.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, forest_grid.predict(X_test))))

Fitting 5 folds for each of 1792 candidates, totalling 8960 fits
Best Parameters : {'bootstrap': True, 'max_depth': 12, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 400}
Best Accuracy Through Grid Search : 0.770

AUC на обучающей выборке: 0.820
AUC на контрольной выборке: 0.767


Выводы: на данном наборе данных, подбор гиперпараметров не обеспечивает существенного прироста

Выполнить оценку важности признаков

In [ ]:
feature_names = X_train.columns
# cоздаем объект со значениями важностей, вычисленными моделью forest
importances = forest.feature_importances_
# задаем сортировку значений важности и сопоставляем названия предикторов важностям
indices = np.argsort(importances)[::-1]

#print("Важность предикторов:")
#for f, idx in enumerate(indices):
#    print("{:2d}. '{:5s}' ({:.4f})".format(f + 1, feature_names[idx], importances[idx]))

График важности для первых 20 признаков:

In [ ]:
d_first = 20
plt.figure(figsize=(12, 5))
plt.title("Feature importances")
plt.bar(range(d_first), importances[indices[:d_first]], align='center')
plt.xticks(range(d_first), np.array(feature_names)[indices[:d_first]], rotation=90)
plt.xlim([-1, d_first]);

### Выполнить обучение и проверку моделей реализующих алгоритмы Наивного Байеса

GaussianNB (Гаусовский наивный байес) реализует гауссовский наивный байесовский алгоритм для классификации

In [18]:
gaus_nb = GaussianNB()

gaus_nb.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, gaus_nb.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, gaus_nb.predict(X_test))))

AUC на обучающей выборке: 0.642
AUC на контрольной выборке: 0.646


MultinomialNB (Мультиномиальный наивный байес) реализует наивный байесовский алгоритм для полиномиально распределенных данных

In [20]:
mult_nb = MultinomialNB()

mult_nb.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, mult_nb.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, mult_nb.predict(X_test))))

AUC на обучающей выборке: 0.715
AUC на контрольной выборке: 0.722


ComplementNB (Дополнение наивного байеса) реализует наивный байесовский алгоритм дополнения (CNB).  
CNB — это адаптация стандартного полиномиального наивного байесовского алгоритма (MNB), который особенно подходит для несбалансированных наборов данных.

In [21]:
cpl_nb = ComplementNB()

cpl_nb.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, cpl_nb.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, cpl_nb.predict(X_test))))

AUC на обучающей выборке: 0.719
AUC на контрольной выборке: 0.727


CategoricalNB (Категориальный Наивный Байес) реализует категориальный наивный алгоритм Байеса для категориально распределенных данных.  
Предполагается, что матрица выборки  кодируется таким образом, что все категории для каждой функции i представлены числами 0,...,Ni-1 где Ni - количество доступных категорий функций i.

In [22]:
cat_nb = CategoricalNB()

cat_nb.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, cat_nb.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, cat_nb.predict(X_test))))

AUC на обучающей выборке: 0.751
AUC на контрольной выборке: 0.757


Выполнить подбор гиперпараметров модели CategoricalNB

In [23]:
n_classes = np.unique(y_train)

params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0, ],
          'fit_prior': [True, False],
          'min_categories': [1, 4, 12, 18, 25, 30],
          'class_prior': [None, [0.1,]* len(n_classes),]
         }

cat_nb_grid = GridSearchCV(CategoricalNB(), param_grid=params, n_jobs=-1, cv=5, verbose=5)
cat_nb_grid.fit(X_train,y_train)

print('Best Parameters : {}'.format(cat_nb_grid.best_params_))
print('Best Accuracy Through Grid Search : {:.3f}\n'.format(cat_nb_grid.best_score_))

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, cat_nb_grid.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, cat_nb_grid.predict(X_test))))

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best Parameters : {'alpha': 0.01, 'class_prior': None, 'fit_prior': True, 'min_categories': 1}
Best Accuracy Through Grid Search : nan

AUC на обучающей выборке: 0.752
AUC на контрольной выборке: 0.759


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Выводы:
даже без соответсвующей подготовки данных алгоритм CategoricalNB  показал лучший результат среди всех Байесовских алгоритмов,  
Использование подбора гиперпараметров улучшает алгоритм не существенно по отношению к базовому варианту 

### Выполнить обучение и проверку модели LinearRegression

In [24]:
lr = LinearRegression(fit_intercept=True)

lr.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, lr.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, lr.predict(X_test))))

print("Model intercept:", lr.intercept_)
print("Model coefs:    ", lr.coef_)

AUC на обучающей выборке: 0.838
AUC на контрольной выборке: 0.841
Model intercept: 1714.8151192682997
Model coefs:     [ 1.24050648e+01 -3.15434872e-02  1.03644252e+00  3.52534119e-02
 -1.32563777e+01  1.27797207e-01 -1.13145122e+00 -3.69309126e-02
  4.02084723e-02 -6.02178928e-03  1.06360532e-02 -5.70998409e-03
 -8.99379865e-02 -3.11686827e-03 -1.68872872e-02  1.25467644e-02
 -1.02658496e-02 -7.23113599e-03  3.10700151e-02 -9.23250668e-03
  2.53873492e-02 -1.26267009e-02 -1.27401355e-02 -6.69375852e-03
  6.67324568e-03  2.76287325e-02 -9.05193473e-02 -4.39952322e-02
  6.92483101e-02  3.76375368e-02 -2.71888740e-02  1.62176526e-02
  1.09712214e-02  2.42360605e-02 -1.30129668e-02 -1.12230937e-02]


Добавим к модели линейной регрессии полиномиальные базисные функции

In [25]:
lrp = make_pipeline(PolynomialFeatures(),
                           LinearRegression())

#X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, train_size=0.5, random_state=42)

lrp.fit(X_train, y_train)

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, lrp.predict(X_train))))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, lrp.predict(X_test))))

AUC на обучающей выборке: 0.865
AUC на контрольной выборке: 0.854


Вывод:
Без подготовки параметров алгоритм Линейной регресии показывает результат сравнимый с алгоритмом RandomForest,  
при этом использование полиномиальных базисных функций заментно улучшает точность модели 